# Run a test calculation with `a-bam-p`/`acd-1-p` to find gaps in our workflow...

In [1]:
%load_ext autoreload
%autoreload 2

import os as os
import glob as glob
import numpy as np

import parmed as pmd
from paprika import align
from paprika import dummy
from paprika import build

import paprika
print(paprika.__version__)

import smirnovert
print(smirnovert.__version__)

from smirnovert.convert import *

# These versions are coming from the LOCAL git directory -- that's not what we want.

74eb316
Using `git` to find current version`.
74eb316


In [5]:
prefix = './benchmarksets/input_files/'
suffix = '/prmtop-rst7/'
benchmarksets = [
    'cd-set1'
    ]

systems = []

for system in benchmarksets:
    for index, file in enumerate(sorted(glob.glob(prefix + system + suffix + 'acd-1-p.prmtop'))):
        systems.append(file)

## Load in the reference coordinates and SMIRNOFF99Frosst parameters

In [6]:
destination = './benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/'
name = 'smirnoff'
reference_prmtop = 'acd-1-p.prmtop'
reference_inpcrd = 'acd-1-p.rst7'
host_resname = 'MGO'
guest_resname = 'MOL'

nonunique = convert(destination, name, 
    reference_prmtop, reference_inpcrd, 
    host_resname, guest_resname,
               debug=True)

INFO:root:Creating ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/smirnoff.pdb with CONECT records...
DEBUG:root:PDB file written by cpptraj.
INFO:root:Pruning water-water CONECT records...
DEBUG:root:First water residue = 151
DEBUG:root:Found first water CONECT entry at line = 6304
INFO:root:Splitting topology into components...
INFO:root:Creating a combined topology for the host and guest molecules...
INFO:root:Writing a `mol2` for the host molecule...
DEBUG:root:MOL2 file written by cpptraj.
INFO:root:Writing a `mol2` for the host molecule...
DEBUG:root:MOL2 file written by cpptraj.
INFO:root:Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/MGO.mol2 to SYBYL atom types via Antechamber...
DEBUG:root:MOL2 file written by antechamber.
INFO:root:Converting ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/MOL.mol2 to SYBYL atom types via Antechamber...
DEBUG:root:MOL2 file written by antechamber.
INFO:root:Extracting water and 

In [7]:
from smirnovert.amber import create_mapping

In [8]:
host_mapping, guest_mapping = create_mapping(nonunique, host_resname, guest_resname)

In [9]:
from smirnovert.amber import remap_atom_types

In [10]:
parm = pmd.amber.LoadParm(destination + 'hg.prmtop')

In [11]:
remap_atom_types(parm, host_resname, host_mapping, guest_resname, guest_mapping, destination)

Assigning 0 C1 1 → C0
Assigning 0 H1 2 → H0
Assigning 0 O1 3 → O0
Assigning 0 C2 4 → C1
Assigning 0 H2 5 → H1
Assigning 0 O2 6 → O1
Assigning 0 HO2 7 → H2
Assigning 0 C3 8 → C2
Assigning 0 H3 9 → H3
Assigning 0 O3 10 → O2
Assigning 0 HO3 11 → H4
Assigning 0 C4 12 → C3
Assigning 0 H4 13 → H5
Assigning 0 C5 14 → C4
Assigning 0 H5 15 → H6
Assigning 0 O5 16 → O3
Assigning 0 C6 17 → C5
Assigning 0 H61 18 → H7
Assigning 0 H62 19 → H8
Assigning 0 O6 20 → O4
Assigning 0 HO6 21 → H9
Assigning 1 C1 22 → C0
Assigning 1 H1 23 → H0
Assigning 1 O1 24 → O0
Assigning 1 C2 25 → C1
Assigning 1 H2 26 → H1
Assigning 1 O2 27 → O1
Assigning 1 HO2 28 → H2
Assigning 1 C3 29 → C2
Assigning 1 H3 30 → H3
Assigning 1 O3 31 → O2
Assigning 1 HO3 32 → H4
Assigning 1 C4 33 → C3
Assigning 1 H4 34 → H5
Assigning 1 C5 35 → C4
Assigning 1 H5 36 → H6
Assigning 1 O5 37 → O3
Assigning 1 C6 38 → C5
Assigning 1 H61 39 → H7
Assigning 1 H62 40 → H8
Assigning 1 O6 41 → O4
Assigning 1 HO6 42 → H9
Assigning 2 C1 43 → C0
Assigning 

## Now re-load in the new structure

In [12]:
hg = pmd.load_file(destination + 'smirnoff-unique.prmtop', destination + 'hg.inpcrd', structure=True)

## Align

In [13]:
# Find the guest atoms to align by reading first line in /data/nhenriksen/projects/cds/wat6/bgbg-tip3p/pdbs
hg = align.zalign(hg, ':MOL@C4', ':MOL@N1', save=True, filename=destination + 'hg-aligned.pdb')

INFO:root:Moving :MOL@C4 (1 atoms) to the origin...
INFO:root:Aligning :MOL@N1 (1 atoms) with the z axis...
INFO:root:Saved aligned coordinates to ./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/hg-aligned.pdb


## Add dummy atoms

In [14]:
hg = pmd.load_file(destination + 'hg-aligned.pdb', structure=True)
# print(hg.atoms[-1].number)

hg = dummy.add_dummy(hg, residue_name='DM1', z=-6.000)
hg = dummy.add_dummy(hg, residue_name='DM2', z=-9.000)
hg = dummy.add_dummy(hg, residue_name='DM3', z=-11.200, y=2.200)

dummy.write_dummy_frcmod(path = destination)
dummy.write_dummy_mol2(path = destination, residue_name='DM1', filename='DM1.mol2')
dummy.write_dummy_mol2(path = destination, residue_name='DM2', filename='DM2.mol2')
dummy.write_dummy_mol2(path = destination, residue_name='DM3', filename='DM3.mol2')

hg.write_pdb(destination + 'hg-aligned-dum.pdb', renumber=False)

## Solvate

In [17]:
base_name = 'smirnoff'
with open(destination + 'tleap.in', 'w') as file:
    tleap_header = f'''\n
source leaprc.water.tip3p
loadamberparams {base_name}-unique.frcmod
{host_resname} = loadmol2 {base_name}-{host_resname}-unique.mol2
{guest_resname} = loadmol2 {base_name}-{guest_resname}-unique.mol2

DM1 = loadmol2 DM1.mol2
DM2 = loadmol2 DM2.mol2
DM3 = loadmol2 DM3.mol2
loadamberparams dummy.frcmod

model = loadpdb {str('hg-aligned-dum.pdb')}
bond model.1.O1 model.2.C4
bond model.2.O1 model.3.C4
bond model.3.O1 model.4.C4
bond model.4.O1 model.5.C4
bond model.5.O1 model.6.C4
bond model.6.O1 model.1.C4


'''
    file.write(tleap_header)
#     for line in bonds_between_monomers:
#         file.write(line + '\n')

In [23]:
destination = './benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/'

build.solvate(tleap_file='tleap.in', pdb_file='hg-aligned-dum.pdb', pbc_type='rectangular', 
              buffer_target=2000,
              output_prefix='hg-solvated', path=destination)

DEBUG:root:Cycle	Buffer Waters Target
DEBUG:root:00	 1 748 (2000)
DEBUG:root:01	 11 1534 (2000)
DEBUG:root:02	 21 2329 (2000)
DEBUG:root:03	 16 1936 (2000)
DEBUG:root:04	 16 1988 (2000)
DEBUG:root:05	 16 1988 (2000)
DEBUG:root:06	 16 2000 (2000)


## Kinda clunky: read in existing file, get mapping, and write out atom name and residue number of restraints

In [22]:
niel = pmd.load_file('./tests/original/a-bam-p/full.topo', './tests/original/a-bam-p/full.crds',
                    structure=True)
dave = pmd.load_file('./benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/hg-solvated.prmtop', 
                     './benchmarksets/input_files/cd-set1/prmtop-rst7/smirnoff/acd-1-p/hg-solvated.rst7',
                    structure=True)

In [23]:
niel_disang = './tests/original/a-bam-p/disang.rest'

In [24]:
lines = []
with open(niel_disang, 'r') as disang:
    for line in disang:
        lines.append(line)

In [25]:
restraint_indices = []
for line in lines[1:]:
    old_restraint_list = line.split()[2]
    old_restraint_indices = [
        int(i) for i in old_restraint_list.split(',') if i is not ''
]
    restraint_indices.append(old_restraint_indices)

In [50]:
niel_host_first_residue = niel[':MGO'].residues[0].number
niel_guest_residue = niel[':BAM'].residues[0].number
niel_dummy_first_residue = niel[':DUM'].residues[0].number

In [64]:
for restraint in restraint_indices:
    names = []
    res_nums = []
    
    offsets = []
    
    for index in restraint:
        name = niel['@{}'.format(index)].atoms[0].name
        res_num = niel['@{}'.format(index)].residues[0].number + 1
        names.append(name)
        res_nums.append(res_num)
        
        if res_num < 4:
            dummy_offset = res_num - niel_dummy_first_residue
        
        if res_num < 10:
            host_offset = res_num - niel_host_first_residue
        
    new_restraints = [f':{i}@{j}' for i, j in zip(res_nums, names)]
    explanation = [f'{mapping[i]}@{j}' for i, j in zip(res_nums, names)]
    remapped_restraints = [f'{remapping[mapping[i]]}@{j}' for i, j in zip(res_nums, names)]
    print(f'{restraint} → {new_restraints} → {explanation} → {remapped_restraints}')

[1, 13] → [':1@Pb', ':4@O3'] → ['D1@Pb', 'H1@O3'] → ['8@Pb', '1@O3']
[2, 1, 13] → [':2@Pb', ':1@Pb', ':4@O3'] → ['D2@Pb', 'D1@Pb', 'H1@O3'] → ['9@Pb', '8@Pb', '1@O3']
[3, 2, 1, 13] → [':3@Pb', ':2@Pb', ':1@Pb', ':4@O3'] → ['D3@Pb', 'D2@Pb', 'D1@Pb', 'H1@O3'] → ['10@Pb', '9@Pb', '8@Pb', '1@O3']
[1, 13, 46] → [':1@Pb', ':4@O3', ':6@C1'] → ['D1@Pb', 'H1@O3', 'H3@C1'] → ['8@Pb', '1@O3', '3@C1']
[2, 1, 13, 46] → [':2@Pb', ':1@Pb', ':4@O3', ':6@C1'] → ['D2@Pb', 'D1@Pb', 'H1@O3', 'H3@C1'] → ['9@Pb', '8@Pb', '1@O3', '3@C1']
[1, 13, 46, 104] → [':1@Pb', ':4@O3', ':6@C1', ':8@C6'] → ['D1@Pb', 'H1@O3', 'H3@C1', 'H5@C6'] → ['8@Pb', '1@O3', '3@C1', '5@C6']
[1, 143] → [':1@Pb', ':10@C4'] → ['D1@Pb', 'G1@C4'] → ['8@Pb', '7@C4']
[2, 1, 143] → [':2@Pb', ':1@Pb', ':10@C4'] → ['D2@Pb', 'D1@Pb', 'G1@C4'] → ['9@Pb', '8@Pb', '7@C4']
[1, 143, 130] → [':1@Pb', ':10@C4', ':10@N1'] → ['D1@Pb', 'G1@C4', 'G1@N1'] → ['8@Pb', '7@C4', '7@N1']
[19, 4, 6, 36] → [':4@O5', ':4@C1', ':4@O1', ':5@C4'] → ['H1@O5', 'H1@C1',

In [52]:
niel.residues[0:10]

[<Residue DUM[0]>,
 <Residue DUM[1]>,
 <Residue DUM[2]>,
 <Residue MGO[3]>,
 <Residue MGO[4]>,
 <Residue MGO[5]>,
 <Residue MGO[6]>,
 <Residue MGO[7]>,
 <Residue MGO[8]>,
 <Residue BAM[9]>]

In [56]:
mapping = {
    1: 'D1',
    2: 'D2',
    3: 'D3',
    4: 'H1',
    5: 'H2',
    6: 'H3',
    7: 'H4',
    8: 'H5',
    9: 'H6',
    10: 'G1'
}

In [10]:
dave.residues

ResidueList([
	<Residue MGO[0]>
	<Residue MGO[1]>
	<Residue MGO[2]>
	<Residue MGO[3]>
	<Residue MGO[4]>
	<Residue MGO[5]>
	<Residue MOL[6]>
	<Residue DM1[7]>
	<Residue DM2[8]>
	<Residue DM3[9]>
	<Residue Cl-[10]>
	<Residue WAT[11]>
	<Residue WAT[12]>
	<Residue WAT[13]>
	<Residue WAT[14]>
	<Residue WAT[15]>
	<Residue WAT[16]>
	<Residue WAT[17]>
	<Residue WAT[18]>
	<Residue WAT[19]>
	<Residue WAT[20]>
	<Residue WAT[21]>
	<Residue WAT[22]>
	<Residue WAT[23]>
	...
	<Residue WAT[2006]>
	<Residue WAT[2007]>
	<Residue WAT[2008]>
	<Residue WAT[2009]>
	<Residue WAT[2010]>
])

In [61]:
remapping = {
    'H1' : 1,
    'H2' : 2,
    'H3' : 3,
    'H4' : 4,
    'H5' : 5,
    'H6' : 6,
    'G1' : 7,
    'D1' : 8,
    'D2' : 9,
    'D3' : 10
}

The algorithm should be:

1. Find the first host residue.
2. Rewrite all of Niel's restraints in terms of dummy atom residues, the guest residue, and the first host residue.

In [45]:
indices = [i + 1 for i in pmd.amber.mask.AmberMask(dave, ':DM1').Selected()]

In [46]:
indices

[144]